In [8]:
import ipychart as ipyc

In [9]:
chart = ipyc.BarChart()
chart.set_data([10, 20, 30, 40, 50, 80])
chart

BarChart(layout=Layout(align_self='stretch', height='auto'))

In [22]:
import ipywidgets as widgets
from traitlets import Unicode, default, List

class DevChart(widgets.DOMWidget):
    """An example widget."""
    _view_name = Unicode('DevChartView').tag(sync=True)
    _view_module = Unicode('dev').tag(sync=True)
    _model_data = List([]).tag(sync=True)
    _type = Unicode().tag(sync=True)
    
    def __init__(self, data, kind):
        
        super().__init__()
        assert kind in ['line', 'bar', 'radar', 'doughnut', 'polarArea', 'bubble'], 'Type must be one of : line, bar, radar, doughnut, polarArea, bubble'
        assert isinstance(data, (list, dict)), 'Please enter data as dict of list'
        
        self._model_data = data
        self._type = kind

    @default('layout')
    def _default_layout(self):
        return widgets.Layout(height='auto', align_self='stretch')
    

In [23]:
%%javascript
require.undef('dev');


define('dev', ["@jupyter-widgets/base", "./js/node_modules/chart.js/dist/Chart.min.js"], function(widgets) {
    var DevChartView = widgets.DOMWidgetView.extend({
    render: function() {
        
        var data = this.model.get("_model_data");
        var type = this.model.get("_type");
        console.log(data)
        console.log(type)
        
        let canvas = document.createElement("canvas");
        var ctx = canvas.getContext('2d');
        
        new Chart(ctx, {
            type: type,
            data: {
                labels: ['Red', 'Blue', 'Yellow', 'Green', 'Purple', 'Orange'],
                datasets: [{
                    label: '# of Votes',
                    data: data,
                    backgroundColor: [
                        'rgba(255, 99, 132, 0.2)',
                        'rgba(54, 162, 235, 0.2)',
                        'rgba(255, 206, 86, 0.2)',
                        'rgba(75, 192, 192, 0.2)',
                        'rgba(153, 102, 255, 0.2)',
                        'rgba(255, 159, 64, 0.2)'
                    ],
                    borderColor: [
                        'rgba(255, 99, 132, 1)',
                        'rgba(54, 162, 235, 1)',
                        'rgba(255, 206, 86, 1)',
                        'rgba(75, 192, 192, 1)',
                        'rgba(153, 102, 255, 1)',
                        'rgba(255, 159, 64, 1)'
                    ],
                    borderWidth: 1
                }]
            },
            options: {
                scales: {
                    yAxes: [{
                        ticks: {
                            beginAtZero: true
                        }
                    }]
                }
            }
        });
        this.el.appendChild(canvas);
        }
    });
    
    return {
        DevChartView : DevChartView
    };
});

<IPython.core.display.Javascript object>

In [29]:
test_wd = DevChart([12, 19, 3, 5, 2, 3], 'polarArea')
test_wd

DevChart(layout=Layout(align_self='stretch', height='auto'))